In [18]:
%run "utilities.ipynb"
from IPython.display import clear_output
clear_output()

ModuleNotFoundError: No module named 'SchemDraw'

In [33]:
import time, re, collections

def myred(x):
    CRED = '\033[91m'
    CEND = '\033[0m'
    return CRED+x+CEND

def mygreen(x):
    CGREEN= '\033[32m'
    CEND = '\033[0m'
    return CGREEN+x+CEND

class QuitExecution(Exception):
    pass

class HaltExecution(Exception):
    pass

def extractvars(src):
    varnames = re.findall( r'(?<!f)[a-zA-Z\_0-9]+', src, re.I | re.M)
    arrays = ["X","X_nonblank","Y","Y_nonblank"]
    scalars = []
    for t in varnames:
        if t in ['i','0','1'] or t=='NAND': continue
        if t[0].isupper():
            if not t in arrays: arrays.append(t)
        else:
            if not t in scalars: scalars.append(t)
    return scalars, arrays
    
class NANDTM:
   
        
    def _repr_pretty_(self, p, cycle):
        if cycle: return "cycle"
        print (self.source)
        
    def __init__(self, source):
        self.source = [l for l in source.split('\n') if l]
        self.scalars , self.arrays  =  extractvars(source)
        self.vals = collections.defaultdict(int)
        self.i = 0
        self.pc = 0
        self.maxlen = 1
        self.namelen = max([len(a) for a in self.scalars]+[len(a) for a in self.arrays])
        self.MAXSTEPS = 800
        self.modified = ()
        
    def getval(self,varname, i = 0):
        return self.vals[(varname,i)]
        
    def setval(self,varname,i,val):
        self.vals[(varname,i)] = val
        
    
    def input(self, x):
        for i,a in enumerate(x):
            self.setval("X",i,int(a))
            self.setval("X_nonblank",i,1)
        self.pc = 0
        self.i = 0
        self.maxlen = len(x)
    
    def modifiedvar(self):
        line = self.source[self.pc]
        if line[:9] == "MODANDJMP": return ("_",0)
        foo, op, bar, blah = parseline(line)
        if foo[-1]==']':
            j = foo.find("[")
            name_ = foo[:j]
            i_    = self.i if foo[j+1]=='i' else int(foo[j+1:-1])
        else:
            name_ = foo
            i_ = 0
        return name_,i_
    
    def printstate(self):
        res = ""
        def arrvals(name):
            def v(name,i):
                name_,i_ = self.modifiedvar()
                a = str(self.getval(name,i))
                if (name_,i_)==(name,i):
                    return mygreen(a)
                if self.i == i:
                    return myred(a)
                return a
            return "".join([v(name,i) for i in range(self.maxlen) ])
        name_,i_ = self.modifiedvar()
        res += "i: " + str(self.i) + "\n"
        for a in self.arrays:
            b = mygreen(a.ljust(self.namelen)) if a==name_ else a.ljust(self.namelen)
            res += b + ": "+ arrvals(a)+"\n"
        for a in self.scalars:
            b = mygreen(a.ljust(self.namelen)) if a==name_ else a.ljust(self.namelen)
            res += b + ": "+ str(self.getval(a))+"\n"
        res += "\n"
        for p,l in enumerate(self.source):
            if p == self.pc:
                l = myred(l)
            res += l+"\n"
        print(res)
    
    
    def next(self,printstate = False):
        def pname(foo):
            if foo[-1]==']':
                j = foo.find("[")
                name = foo[:j]
                i    = self.i if foo[j+1]=='i' else int(foo[j+1:-1])
            else:
                name = foo
                i = 0
            return name,i
        
        line = self.source[self.pc]
        if line[:9] == "MODANDJMP":
            j = line.find("(")
            k = line.find(",")
            l = line.find(")")
            a = self.getval(*pname(line[j+1:k].strip()))
            b = self.getval(*pname(line[k+1:l].strip()))
            if printstate:
                clear_output()
                self.printstate()
            if not a and not b: raise HaltExecution("halted")
            if b:
                if a: self.i += 1
                else: self.i = max(0,self.i-1)
            self.pc = 0
            self.maxlen = max(self.maxlen, self.i+1)
            return
        foo, op, bar, blah = parseline(line)
        a = self.getval(*pname(bar))
        b = self.getval(*pname(blah))
        self.setval(*pname(foo),1-a*b)
        self.pc = self.pc + 1 
        if printstate:
            clear_output()
            self.printstate()
    
    def run(self,iterate = False, maxsteps = 0):
        if iterate:
            print("q(uit), n(ext),p(rev),c(lear),r(un),s(kip) XX")
        if not maxsteps:
            maxsteps = self.MAXSTEPS
        t = 0
        noprinting = 0
        quit_cmd = False
        try:
            while True:
                if noprinting>0:
                    noprinting -= 1
                else:
                    clear_output()
                    self.printstate()
                if iterate and noprinting<=0:
                    cmd = input("")
                    #CURSOR_UP_ONE = '\x1b[1A'
                    #ERASE_LINE = '\x1b[2K'
                    #print(CURSOR_UP_ONE + ERASE_LINE + CURSOR_UP_ONE)
                    
                    c = cmd[0] if cmd else "n"
                    if c=="c":
                        clear_output()
                    elif c=="r":
                        iterate = False
                    elif c=="s":
                        _,num = cmd.split(' ')
                        print("...")
                        noprinting = int(num)
                    elif c=="p":
                        self.prev()
                        t -= 1
                        continue
                    elif c=="q":
                        raise QuitExecution("User quit")
                self.next()
                if t >= maxsteps:
                    raise Exception("Too many steps")
                t += 1
               
        except HaltExecution as e:
            msg = str(e)
            clear_output()
            self.printstate()
            y = ""
            i = 0
            while self.getval("Y_nonblank",i) :
                y += str(self.getval("Y",i))
                i += 1
            return y
        except  QuitExecution as e:
            print(str(e))
            

In [66]:
source = r'''temp_0 = NAND(X[0],X[0])
Y_nonblank[0] = NAND(X[0],temp_0)
temp_2 = NAND(X[i],Y[0])
temp_3 = NAND(X[i],temp_2)
temp_4 = NAND(Y[0],temp_2)
Y[0] = NAND(temp_3,temp_4)
MODANDJMP(X_nonblank[i],X_nonblank[i]'''
andprog = NANDTM(source)
andprog.input("1110")

In [67]:
andprog.printstate()

i: 0X         : 1110
X_nonblank: 1111
Y         : 0000
Y_nonblank: 0000
MODANDJMP : 0000
temp_0    : 0
temp_2    : 0
temp_3    : 0
temp_4    : 0

temp_0 = NAND(X[0],X[0])
Y_nonblank[0] = NAND(X[0],temp_0)
temp_2 = NAND(X[i],Y[0])
temp_3 = NAND(X[i],temp_2)
temp_4 = NAND(Y[0],temp_2)
Y[0] = NAND(temp_3,temp_4)
MODANDJMP(X_nonblank[i],X_nonblank[i]



In [65]:
andprog.next()
andprog.printstate()

i: 0X         : 111
X_nonblank: 111
Y         : 000
Y_nonblank: 100
MODANDJMP : 000
temp_0    : 0
temp_2    : 0
temp_3    : 1
temp_4    : 1

temp_0 = NAND(X[0],X[0])
Y_nonblank[0] = NAND(X[0],temp_0)
temp_2 = NAND(X[i],Y[0])
temp_3 = NAND(X[i],temp_2)
temp_4 = NAND(Y[0],temp_2)
Y[0] = NAND(temp_3,temp_4)
MODANDJMP(X_nonblank[i],X_nonblank[i]

